In [1]:
import numpy as np
import pandas as pd

import tensorflow_decision_forests as tfdf

import warnings
warnings.filterwarnings("ignore")

In [2]:
def score(res,label,pre):
    res['diff'] = abs((res[pre]-res[label])/7)
    s = 1-sum(res['diff'])/len(res)
    return s

def get_label(row):
    max_date = row['date_max']
    min_date = row['date_min']
    date = row['date']
    date_list = row['date_list']
    
    if date+7 <= max_date:
        return sum([1 for x in set(date_list) if date < x < date+8])
    else:
        if date>153:
            if row['user_id'] not in user_enddate:
                return -999
            else:
                if user_enddate[row['user_id']] < date+7:
                    return -999
                else:
                    return sum([1 for x in set(date_list) if date < x < date+8])
        elif date<130:
            return -999
        else:
            return sum([1 for x in set(date_list) if date < x < date+8])
        
def extend_list(row):
    date_min = row['date_min']
    date_max = row['date_max']
    
    if row['user_id'] in user_enddate:
        end_date = user_enddate[row['user_id']]
        return list(set([x for x in range(date_min-1, end_date-6)]+interval))
    else:
        return list(set([x for x in range(date_min-1, date_max-6)]+interval))
    
    
def get_date_list(row):
    ls = row['date_list']
    end_date = row['date']
    
    ls_new = [x for x in ls if x<=end_date]
    return ls_new

In [3]:
interval = [x for x in range(131,154)]

In [4]:
df = pd.read_csv('data/app_launch_logs.csv')
test_a = pd.read_csv('data/test_B数据集.csv')

test = pd.read_csv('data/test_B数据集.csv')
test.columns = ['user_id','date']

In [5]:
df = df.sort_values(['user_id','date']).reset_index(drop=True)
df = df[['user_id','date']].drop_duplicates().reset_index(drop=True)
df = df[df['user_id'].isin(test_a['user_id'])]

In [6]:
df_group = df.groupby('user_id').agg(list).reset_index()
df_group['date_max'] = df_group['date'].apply(lambda x: max(x))
df_group['date_min'] = df_group['date'].apply(lambda x: min(x))
user_enddate = dict(zip(test_a['user_id'],test_a['end_date']))

In [7]:
df_group['date_all'] = df_group.apply(extend_list, axis=1)

In [8]:
df_ = df_group.explode('date_all')
df_.rename(columns = {'date':'date_list','date_all':'date'},inplace=True)

In [9]:
df_['label'] = df_.apply(get_label, axis=1)

In [10]:
train = df_[df_['label']!=-999]

In [11]:
train_pb = pd.read_csv('features/online_train_pb.csv')
train = pd.merge(train, train_pb, how='left')

test_pb = pd.read_csv('features/online_test_pb.csv')
test = pd.merge(test, test_pb, how='left')

In [12]:
user_trait = pd.read_csv('features/user_trait_feature.csv')

train = pd.merge(train, user_trait, how='left')
test = pd.merge(test, user_trait, how='left')

In [13]:
launch_train = pd.read_csv('features/launch_online_train.csv')
train = pd.merge(train, launch_train, how='left')

launch_test = pd.read_csv('features/launch_online_test.csv')
test = pd.merge(test, launch_test, how='left')

In [14]:
train['week'] = train['date'].apply(lambda x: (x-130)%7+1)
test['week'] = test['date'].apply(lambda x: (x-130)%7+1)

In [29]:
feats = ['playtime_last0', 'video_count_last0', 'playtime_last1', 'video_count_last1', 'playtime_last2', 'video_count_last2',
         'playtime_last3', 'video_count_last3', 'playtime_last4', 'video_count_last4', 'playtime_last5', 'video_count_last5',
         'playtime_last6', 'video_count_last6', 'playtime_last7', 'video_count_last7', 'device_type', 'sex','age', 'education',
         'occupation_status','device_ram_new','device_rom_new','diff_near','is_launch','launch_type_new',
         'launchNum','NumLastWeek','preds_median_30',
         'preds_mean_4','preds_mean_4_weighted','weighted_median','week']

In [30]:
train_df = train[feats+['label']]
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_df, label="label",task = tfdf.keras.Task.REGRESSION)

2022-01-22 16:24:26.676240: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-22 16:24:26.709396: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-22 16:24:26.709731: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-22 16:24:26.710670: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [31]:
model = tfdf.keras.GradientBoostedTreesModel(task = tfdf.keras.Task.REGRESSION, max_depth = 6, num_trees=500, min_examples=500,
                                    l1_regularization=0.1, l2_regularization=0.1, subsample=0.8)
model.fit(train_ds)

Use /tmp/tmp_q7nm_1k as temporary training directory
Starting reading the dataset
3210/3214 [============================>.] - ETA: 0s
Dataset read in 0:00:24.924314
Training model
Model trained in 0:30:28.543561
Compiling model
3214/3214 [==============================] - 1854s 576ms/step


In [32]:
test['label'] = -1
test_df = test[feats+['label']]
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_df, label="label",task = tfdf.keras.Task.REGRESSION)

In [33]:
y_pred = model.predict(test_ds)
test['pre_gbt'] = y_pred.squeeze()

In [34]:
test[['user_id','pre_gbt']].to_csv('res/tfdf_gbt_online.csv',index=False)